# 12.3 메서드 연결 기법
- 이번 챕터는 이론 위주의 내용이다.

In [3]:
# 데이터셋을 여러 차례 변형해야 하는 경우 분석에는 전혀 필요없는 임시 변수를 계속 생성하는 상황이 발생한다.
# 다음 예제를 살펴보자.

In [5]:
# df = load_data()
# df2 = df[df["col2"] < 0]
# df2["col1_demeaned"] = df2["col1"] - df2["col1"].mean()
# result = df2.groupby("key").col1_demeaned.std()

In [6]:
# 여기서 실제 데이터를 사용하지는 않지만 새로운 메서드 몇 가지를 만나게 되는데 그중 하나는 df[k] = v처럼 컬럼에 값을 대입하는 함수형 DataFrame.assign 메서드다.
# 객체를 변경하는 대신 값 대입이 완료된 새로운 DataFrame을 반환한다.
# 아래 두 코드는 동일하다.

In [7]:
# 실용적이지 않은 방법
# df2 = df.copy()
# df2["k"] = v

# 실용적인 방법
# df2 = df.assign(k=v)

In [8]:
# 값을 직접 대입하는 것이 assign을 사용하는 것보다 빠르게 수행되지만 assign을 이용하면 메서드를 연결해서 사용할 수 있다.

In [9]:
# result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
#           .groupby("key")
#           .col1_demeaned.std())

In [10]:
# 여기서는 줄바꿈을 편리하게 하기 위해 위 코드를 괄호로 둘러쌌다.

In [11]:
# 메서드를 연결해서 사용할 때 주의해야 할 점은 임시 객체를 참조해야 할 경우가 있을 수도 있다는 것이다.
# 앞선 예제에서 load_data의 반환값을 임시 변수인 df에 담기 전까지는 그 결과를 참조할 수 없었다.
# 이런 경우 assign이나 호출이 가능한 객체 또는 함수를 인자로 받는 pandas의 다른 함수를 이용해서 잘 해결할 수 있다.

In [12]:
# 호출이 가능한 객체의 예시를 보기 위해 위 예제의 일부 코드를 다시 살펴보자.

In [13]:
# df = load_data()
# df2 = df[df["col2"] < 0]

In [14]:
# 위 코드는 다음과 같이 고쳐 쓸 수 있다.

# df = (load_data()
#      [lambda x: x["col2"] < 0])

In [15]:
# 여기서 load_data의 결과를 변수에 저장하지 않았다.
# 그래서 []에 함수를 전달해서 메서드 연결이 이어지도록 했다.

# 계속해서 전체 코드를 하나의 메서드 연결 표현으로 작성할 수도 있다.

In [16]:
# result = (load_data()
#           [lambda x: x.col2 < 0]
#           .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
#           .groupby("key")
#           .col1_demeaned.std())

In [17]:
# 어떤 스타일을 선호하는지는 개인의 취향이지만 코드를 적당히 끊어서 사용하는 것은 가독성 향상에 도움이 된다.

- 12.3.1 pipe 메서드

In [18]:
# pandas의 내장 함수와 방금 살펴본 메서드 연결을 통해 다양한 일을 할 수 있다.
# 하지만 직접 작성한 함수나 다른 서드파티 라이브러리의 함수를 사용해야 하는 경우도 생긴다.
# 이때 pipe 메서드를 사용할 수 있다.

# 다음과 같은 일련의 함수 호출을 생각해보자.

In [19]:
# a = f(df, arg1=v1)
# b = g(a, v2, arg3=v3)
# c = h(b, arg4=v4)

In [20]:
# Series나 DataFrame 객체를 인자로 취하고 반환하는 함수를 사용하는 경우 위 코드를 pipe를 이용해서 아래처럼 고쳐 쓸 수 있다.

In [21]:
# result = (df.pipe(f, arg1=v1)
#           .pipe(g, v2, arg3=v3)
#           .pipe(h, arg4=v4))

In [22]:
# f(df)와 df.pipe(f)는 동일하다. 하지만 pipe는 메서드 연결을 좀 더 쉽게 쓸 수 있도록 해준다.

# pipe를 이용한 유용한 패턴 중 하나는 일련의 연산을 재사용 가능한 함수로 일반화하는 것이다.
# 컬럼에서 그룹 평균을 빼는 과정을 생각해보자.

In [23]:
# g = df.groupby(["key1", "key2"])
# df["col1"] = df["col1"] - g.transform("mean")

In [24]:
# 한 컬럼이 아니라 여러 컬럼에 대해 그룹 평균을 뺄 수 있고 그룹의 키를 쉽게 변경할 수 있기를 바란다면, 
# 또 이 작업을 메서드 연결로도 수행할 수 있기 바란다면 아래 예제를 보자.

In [25]:
# def group_demean(df, by, cols):
#   result = df.copy()
#   g = df.groupby(by)
#   for c in cols:
#       result[c] = df[c] - g[c].transform("mean")
#   return result

In [26]:
# 이제 group_demean 함수를 사용해서 아래처럼 작성할 수 있다.

In [27]:
# result = (df[df.col1 < 1]
#           .pipe(group_demean, ["key1", "key2"], ["col1"]))